In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
import itertools
import glob

Load in Pre-Trained Model

In [ ]:
model = tf.keras.models.load_model('/dbfs/FileStore/tables/Models/vX')

(Option 1): Make Predictions Using Folders of Images

In [ ]:
# Folders follow the structure where there is 1 base folder which contains 1 folder for each label

emotions = ["Happy", "Surprise", "Disgust", "Neutral", "Angry", "Fear", "Sad"]

# Get paths to all of the images to predict on
file_paths = []
for emotion in emotions:
    path = f"/dbfs/FileStore/tables/Test Set Cropped/{emotion}/"
    files = glob.glob(path + "*.jpg", recursive=True)
    file_paths.extend(files)

xdata = []
ydata = []
# Get pixels with corresponding labels for the images
for file_path in file_paths:
    image = Image.open(file_path).convert('L').getdata()
    resized = image.resize((48, 48))
    xdata.append(np.array(resized).reshape((48, 48, 1)))
    ydata.append(file_path.split('/')[5])

xtest = np.array(xdata).astype('float64')
ytest = np.array(ydata)

# Preprocess data the same way as done to the training data
xtest -= np.mean(xtest, axis=0)
xtest /= np.std(xtest, axis=0)

In [ ]:
predicted_probabilities = model.predict(xtest) 

predicted = []
actual = []
emotion_to_int = {"Angry":0, "Disgust":1, "Fear":2, "Happy":3, "Sad":4, "Surprise":5, "Neutral":6}

# Get a single prediction for each list of probabilities
for i in range(len(predicted_probabilities)):
    predicted.append(np.argmax(predicted_probabilities[i]))
    actual.append(emotion_to_int[ytest[i]])

(Option 2): Make Predictions Using Data Stored as a .npy Dataset

In [ ]:
# Load in the data
xtest = np.load('/dbfs/FileStore/tables/npy/xtest.npy')
ytest = np.load('/dbfs/FileStore/tables/npy/ytest.npy').tolist()

# Preprocess data the same way as done to the training data
xtest -= np.mean(xtest, axis=0)
xtest /= np.std(xtest, axis=0)

In [ ]:
predicted_probabilities = model.predict(xtest)

predicted = []
actual = []

for i in range(len(predicted_probabilities)):
    predicted.append(np.argmax(predicted_probabilities[i]))
    actual.append(ytest[i].index(True))

Metrics

In [ ]:
# Display the confusion matrix for the test data

labels = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
cm = confusion_matrix(actual, predicted)
title='Confusion Matrix'
    
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)

plt.colorbar()
tick_marks = np.arange(len(labels))
plt.xticks(tick_marks, labels, rotation=45)
plt.yticks(tick_marks, labels)

fmt = 'd'
thresh = cm.max() / 2. 
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], fmt), 
        horizontalalignment="center",
        color="white" if cm[i, j] > thresh else "black")

plt.ylabel('True')
plt.xlabel('Predicted')
plt.tight_layout()

In [ ]:
print(f"F1-Score:   {dict( zip( labels, f1_score(actual, predicted, average=None) ) )}")
print(f"Accuracy:   {accuracy_score(actual, predicted)}")
print(f"Precision:  {dict( zip( labels, precision_score(actual, predicted, average=None) ) )}")
print(f"Recall:     {dict( zip( labels, recall_score(actual, predicted, average=None) ) )}")